<a href="https://colab.research.google.com/github/neethu-vm/Computer-vision-OpenCV/blob/main/food_recognition_visualization_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyyaml==5.3
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.8") 

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f \https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 268 kB 8.2 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3-cp37-cp37m-linux_x86_64.whl size=44244 sha256=2ef9446ed27c6676ef14dcb9bd5ed17954987c788d576f61e0bd4fca9612d319
  Stored in directory: /root/.cache/pip/wheels/8a/55/a4/c0a81d27c33462cfdcb904db018f5550197e88b2b6b85beed2
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
     |████████████████████████████████| 17.3 MB 177 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successf

In [ ]:
!nvidia-smi

Tue Aug 17 07:40:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# import some common libraries
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os, json, cv2, random, glob
from google.colab.patches import cv2_imshow

In [ ]:
# import some common detectron2 utilities
import os
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# For reading annotations file
from pycocotools.coco import COCO

# utilities
from pprint import pprint # For beautiful print!


# For data visualisation
import plotly.graph_objects as go
import plotly.express as px
from google.colab.patches import cv2_imshow

In [ ]:
TRAIN_ANNOTATIONS_PATH = "/content/drive/MyDrive/visualization/trainval.json"
TRAIN_IMAGE_DIRECTIORY = "/content/drive/MyDrive/visualization/trainimages"

train_coco = COCO(TRAIN_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=1.16s)
creating index...
index created!


In [ ]:
# Reading the annotation files
with open(TRAIN_ANNOTATIONS_PATH) as f:
  train_annotations_data = json.load(f)

In [ ]:
train_annotations_data['annotations'][0]

{'area': 195230.62220210256,
 'bbox': [1.0, 186.0, 577.0, 504.0],
 'category_id': 9,
 'id': 1,
 'image_id': 0,
 'iscrowd': 0,
 'segmentation': [[1.4751773049645465,
   244.6808510638298,
   139.06382978723406,
   197.87234042553192,
   283.0354609929078,
   186.52482269503548,
   434.8085106382979,
   288.65248226950354,
   534.8085106382979,
   426.2411347517731,
   578.7801418439717,
   690.7801418439717,
   499.3475177304965,
   690.7801418439717,
   1.4751773049645465,
   493.6170212765958]]}

In [ ]:
# Reading all classes

category_ids = train_coco.loadCats(train_coco.getCatIds())

category_names = [_["name"] for _ in category_ids]

pprint(", ".join(category_names))

('banana, biriyani, bread, chai, chappathi, chicken curry, chicken fry, '
 'chutney, curry, dosa, egg, egg curry, fish curry, fish fry, idli, kadala '
 'curry, pappadam, pazhampori, pickle, puttu, rice, salad, tea, upma, vada, '
 'veg curry, vellappam')


In [ ]:
# Getting all categoriy with respective to their total images

no_images_per_category = {}

for n, i in enumerate(train_coco.getCatIds()):
  imgIds = train_coco.getImgIds(catIds=i)
  label = category_names[n]
  no_images_per_category[label] = len(imgIds)

img_info = pd.DataFrame(train_coco.loadImgs(train_coco.getImgIds()))

no_images_per_category

{'banana': 79,
 'biriyani': 267,
 'bread': 199,
 'chai': 128,
 'chappathi': 234,
 'chicken curry': 57,
 'chicken fry': 77,
 'chutney': 317,
 'curry': 47,
 'dosa': 332,
 'egg': 10,
 'egg curry': 1,
 'fish curry': 22,
 'fish fry': 71,
 'idli': 314,
 'kadala curry': 96,
 'pappadam': 212,
 'pazhampori': 7,
 'pickle': 153,
 'puttu': 224,
 'rice': 276,
 'salad': 91,
 'tea': 4,
 'upma': 228,
 'vada': 158,
 'veg curry': 193,
 'vellappam': 204}

In [ ]:
pd.DataFrame(no_images_per_category.items()).sort_values(1).iloc[::-1][0][:30].tolist()

['dosa',
 'chutney',
 'idli',
 'rice',
 'biriyani',
 'chappathi',
 'upma',
 'puttu',
 'pappadam',
 'vellappam',
 'bread',
 'veg curry',
 'vada',
 'pickle',
 'chai',
 'kadala curry',
 'salad',
 'banana',
 'chicken fry',
 'fish fry',
 'chicken curry',
 'curry',
 'fish curry',
 'egg',
 'pazhampori',
 'tea',
 'egg curry']

In [ ]:
fig = go.Figure([go.Bar(x=list(no_images_per_category.keys()), y=list(no_images_per_category.values()))])
fig.update_layout(
    title="No of Image per class",)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=list(no_images_per_category.keys()), values=list(no_images_per_category.values()), 
                             hole=.3, textposition='inside', )], )
fig.update_layout(
    title="No of Image per class ( In pie )",)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=img_info['height']))
fig.add_trace(go.Histogram(x=img_info['width']))

# Overlay both histograms
fig.update_layout(barmode='stack', title="Histogram of Image width & height",)


fig.show()

In [ ]:
img_info

,height,width,id,file_name
0,694,640,0,videos(98).png
1,538,720,1,chappathi_img212.jpg
2,720,720,2,biriyani_img208.jpg
3,720,720,3,chappathi_img126.jpg
4,960,720,4,biriyani_img104.jpg
...,...,...,...,...
2652,640,640,2652,60840726_348516505804945_6433218460825681920_n...
2653,959,720,2653,bread_122.jpg
2654,530,720,2654,bread_197.jpg
2655,720,720,2655,idli_128.jpg
